<h1><center><strong>San Francisco Crime Classification</strong></center></h1>
<h3><center>General Prediction</center></h3>
<h3>Data Mining | Fall 2023</h3>

Contributors:
- Kevin Reynolds
- Shivani Merchant
- Kyrsti Fitts
- Ryan Espejo

# Data Preprocessing

In [ ]:
import pandas as pd
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
# Silence warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# Load training data and assign names to attributes
training_attribute_names = ["Dates", "Category", "Descript", "DayOfWeek", "PdDistrict", "Resolution", "Address", "X", "Y"]
training_data = pd.read_csv('data/train.csv', skiprows=1, names=training_attribute_names)

# Remove irrelevant columns
relevant_data = ["Dates", "PdDistrict", "Address", "Category"]
training_data = training_data[relevant_data]
training_data.head()


In [ ]:
# Load training data and assign names to attributes
test_attribute_names = ["Id", "Dates", "DayOfWeek", "PdDistrict", "Address", "X", "Y"]
test_data = pd.read_csv('data/test.csv', skiprows=1, names=test_attribute_names)

# remove duplicates from train and test
training_data.drop_duplicates(inplace=True)

# Remove IDs from test data. We will copy them and use them later for final output
test_ids = test_data['Id'].copy()

# Drop unwanted data
test_data.drop('Id', axis=1, inplace=True)
test_data.drop('DayOfWeek', axis=1, inplace=True)
test_data.drop('X', axis=1, inplace=True)
test_data.drop('Y', axis=1, inplace=True)

test_data.head()

In [ ]:
# Convert Dates column to datetime
training_data['Dates'] = pd.to_datetime(training_data['Dates'])

# Extract hour
training_data['Hour'] = training_data['Dates'].dt.hour
training_data.drop('Dates', axis=1, inplace=True)

training_data.head()

In [ ]:
# Convert Dates column to datetime
test_data['Dates'] = pd.to_datetime(test_data['Dates'])

# Extract Hour
test_data['Hour'] = test_data['Dates'].dt.hour
test_data.drop('Dates', axis=1, inplace=True)

test_data.head()

In [ ]:
# Encode the rest of the columns
encoded_training_data = training_data.copy()
encoded_test_data = test_data.copy()

columns_to_encode = ['Address', 'PdDistrict']

for col in columns_to_encode:
    combined = pd.concat([encoded_training_data[col], encoded_test_data[col]], axis=0).astype(str)
    label_encoder = LabelEncoder().fit(combined)
    
    encoded_training_data[col] = label_encoder.transform(encoded_training_data[col].astype(str))
    encoded_test_data[col] = label_encoder.transform(encoded_test_data[col].astype(str))

encoded_training_data.head()


In [ ]:
encoded_test_data.head()

### Fit Decision Tree

In [ ]:
# Separate targets from training data
X_train = encoded_training_data.drop(columns='Category')
y_train = training_data['Category']

# Encode y_train
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

In [ ]:
# Let it cook
X_train, X_val, y_train_encoded, y_val_encoded = train_test_split(X_train, y_train_encoded, test_size = 0.2, random_state = 0)
model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=100, alpha=1e-4, solver='adam', verbose=10, random_state=1, learning_rate_init=.1)
model.fit(X_train, y_train_encoded)

### Predict with Decision Tree

In [ ]:
# Predict categories for the test data
predicted_categories_encoded = model.predict_proba(encoded_test_data)

## Output

In [ ]:
# Create a DataFrame for the predicted probabilities
num_categories = len(label_encoder.classes_)
predicted_categories = label_encoder.classes_
predicted_probabilities_df = pd.DataFrame(predicted_categories_encoded, columns=predicted_categories)

# Insert 'Id' column
predicted_probabilities_df.insert(0, 'Id', test_ids.values)

# Round the probabilities
predicted_probabilities_df = predicted_probabilities_df.round(2)

# Save to CSV
predicted_probabilities_df.to_csv('predictions/neural_network.csv', index=False, header=True)